In [1]:
# %pip install -q openai python-dotenv

# (Azure) OpenAI Function Calling

## How does Function Calling works

#### Sample Full GPT Response JSON
```json
{
    "id": "chatcmpl-XXXXXXXXXXXXXXX",
    "choices": [
        {
            "finish_reason": "tool_calls",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "refusal": null,
                "role": "assistant",
                "audio": null,
                "function_call": null,
                "tool_calls": [
                    {
                        "id": "call_XXXXXXXXXXXXX",
                        "function": {
                            "arguments": "{\"name\":\"Jhon Doe\"}",
                            "name": "greeting_user"
                        },
                        "type": "function"
                    }
                ]
            },
            "content_filter_results": {}
        }
    ],
    "created": 1732238669,
    "model": "gpt-4o-2024-08-06",
    "object": "chat.completion",
    "service_tier": null,
    "system_fingerprint": "fp_d54531d9eb",
    "usage": {
        "completion_tokens": 16,
        "prompt_tokens": 82,
        "total_tokens": 98,
        "completion_tokens_details": null,
        "prompt_tokens_details": null
    },
    "prompt_filter_results": [
        {
            "prompt_index": 0,
            "content_filter_results": {
                "hate": {
                    "filtered": false,
                    "severity": "safe"
                },
                "jailbreak": {
                    "filtered": false,
                    "detected": false
                },
                "self_harm": {
                    "filtered": false,
                    "severity": "safe"
                },
                "sexual": {
                    "filtered": false,
                    "severity": "safe"
                },
                "violence": {
                    "filtered": false,
                    "severity": "safe"
                }
            }
        }
    ]
}
```

### Environment Set Up

In [36]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from IPython.display import Markdown

load_dotenv()

AOAI_ENDPOINT = os.getenv("AOAI_ENDPOINT")
AOAI_API_KEY = os.getenv("AOAI_API_KEY")
AOAI_API_VERSION = os.getenv("AOAI_API_VERSION")
AOAI_DEPLOYMENT = os.getenv("AOAI_DEPLOYMENT")

client = AzureOpenAI(
  azure_endpoint=AOAI_ENDPOINT, 
  api_key=AOAI_API_KEY, 
  api_version=AOAI_API_VERSION, 
)

### API Definition

##### Function to Call

In [37]:
def greeting_user(name: str) -> str:
  print(f"Hello World!! - {name}")

##### Function Definition Payload

In [31]:
greeting_user_definition = {
  "name": "greeting_user",
  "description": "Greeting ther user by name. Call this whenever you want to greet the user.",
  "parameters": {
    "type": "object",
    "properties": {
      "name": {
        "type": "string",
        "description": "The name to greet"
      }
    },
    "required": ["name"],
    "additionalProperties": False
  }
}

##### Tools Payload

In [32]:
tools = [
  {
    "type": "function",
    "function": greeting_user_definition,
  }
]

##### Messages Payload

In [ ]:
messages = [
  {
    "role": "system",
    "content": "You are a helpful assistant. Use the supplied tools to assist the user."
  },
  {
    "role": "user",
    "content": "Hello!! My name is Jhon Doe."
  }
]

##### Call API

In [7]:
# Call API
response = client.chat.completions.create(
  model=AOAI_DEPLOYMENT,
  messages=messages,
  tools=tools
)

#### Response
If the GPT Model decides to use a Tool, it will be indicated in the API response in the following way:

`response.choices[0].message`

```json
{
    "content": null,
    "refusal": null,
    "role": "assistant",
    "audio": null,
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_L8yIyAnF9rT1U5Rn8LW3sS4p",
            "function": {
                "arguments": "{\"name\":\"Goku\"}",
                "name": "greeting_user"
            },
            "type": "function"
        }
    ]
}
```

The `tool_calls` property will contain the name of the function to execute and the parameters.

#### Call the Function

In [38]:
import json

message = response.choices[0].message

if len(message.tool_calls) > 0:
  tool = message.tool_calls[0]

  function_name = tool.function.name
  arguments = json.loads(tool.function.arguments)
  
  function = globals()[function_name]
  
  function(**arguments)

Hello World!! - Goku
